Nous avons tout d'abord créé un fichier function.py contenant les différentes fonctions qui seront utiles.

Installation des dépendances :

In [19]:
pip install openai fastapi pandas joblib scikit-learn

  Using cached openai-1.33.0-py3-none-any.whl (325 kB)


Imports :

In [20]:
from dotenv import load_dotenv
import os
from openai import OpenAI
from fastapi import HTTPException
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [22]:
load_dotenv()

client = OpenAI(
    api_key="<API_KEY>"
)

Nous avons créer une fonction permettant de faire un appel à OpenAI pour générer une réponse à l'aide d'un champ texte.



In [23]:
def callOpenAI(text: str):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Tu es un chatbot qui répond à des questions sur les joueurs de football, tu as accès à une base de données sur les joueurs de football. Quand tu parles de Lionel Messi, tu veux dire Pionel Pessi."},
            {"role": "user", "content": "Qui est le meilleur joueur de football ?"},
            {"role":"assistant", "content":"Le meilleur joueur de football est Pionel Pessi."},
            {"role": "user", "content": text}
        ]
    )

    return response.choices[0].message.content

Fonction permettant d'entrainer un modèle de machine learning pour les résultats de match de foot.

In [24]:
def train(data: pd.DataFrame):

    features = data[['home_team', 'away_team']]
    target = data['home_win']

    # Encodage des données en variables numériques
    features = pd.get_dummies(features)

    # Séparation des ensembles de formation et de test
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.22)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    model = LogisticRegression(max_iter=1000)

    # Entraînement du modèle
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # Évaluation du modèle
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\nTest Accuracy: {accuracy*100:.2f}%")

    joblib.dump({"model": model, "scaler": scaler, "features": features}, "model")


    return accuracy

La fonction predict utilise un modèle d'apprentissage automatique pré-entraîné pour prédire l'issue d'un match de football en fonction des équipes données


In [25]:
def predict(teams: dict):
    # Nom des équipes pour la prédiction
    home_team = teams['team1']
    away_team = teams['team2']

    # Chargement du modèle
    model_data = joblib.load("model")
    model = model_data["model"]
    scaler = model_data["scaler"]
    features = model_data["features"]

    new_match = pd.DataFrame({'home_team': [home_team], 'away_team': [away_team]})

    new_match_encoded = pd.get_dummies(new_match)
    new_match_encoded = new_match_encoded.reindex(columns=features.columns, fill_value=0)

    new_match_normalized = scaler.transform(new_match_encoded)

    # Prédiction de l'issue du match
    prediction = model.predict(new_match_normalized)
    prediction_proba = model.predict_proba(new_match_normalized)

    if prediction[0] == 1:
        print(f"{home_team} va probablement gagner.")
        return {"winner": home_team, "prediction_score": prediction_proba[0][1]}
    else:
        print(f"{away_team} va probablement gagner.")
        return {"winner": away_team, "prediction_score": prediction_proba[0][0]}

Nous avons créé un fichier api.py contenant l'API (FastAPI).



Installation des dépendances

In [13]:
pip install fastapi uvicorn

Import de FastAPI, de pandas...

In [27]:
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware
import pandas as pd
import io

In [28]:
app = FastAPI()
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

Endpoint pour l'entrainement du modèle en utilisant un csv fournit par l'utilisateur.

In [29]:
@app.post("/training")
async def create_training(file: UploadFile = File(...)):
    content = await file.read()

    if not file.filename.endswith('.csv'):
        raise HTTPException(status_code=400, detail="Le fichier doit être un fichier CSV.")

    df = pd.read_csv(io.StringIO(content.decode('utf-8')))

    if 'home_team' not in df.columns or 'away_team' not in df.columns or 'home_win' not in df.columns:
        raise HTTPException(status_code=400, detail="Les colonnes 'home_team', 'away_team' et 'home_win' sont requises.")

    response = train(df)
    return {"score": response}

Endpoint pour la prédiction des matchs de foots basés sur le nom des deux équipes

In [30]:
@app.post("/predict")
async def predict_match(teams: dict):
    if 'team1' not in teams or 'team2' not in teams:
        raise HTTPException(status_code=400, detail="Les noms des équipes 'team1' et 'team2' sont requis.")
    response = predict(teams)
    return response

Endpoint pour intéragir avec openai



In [33]:
@app.get("/model")
async def get_model(text: str):
    if not text:
        raise HTTPException(status_code=400, detail="Le texte est requis.")
    response = callOpenAI(text)
    return response